In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!git pull

Already up to date.


In [5]:
! pip install mlflow transformers datasets "pydantic[dotenv]" av

In [6]:
import mlflow
from mlflow.client import MlflowClient
import pandas as pd
import os
import matplotlib.pyplot as plt
from overfit.env_settings import settings
from overfit.utils.mlflow import get_all_experiments, get_params_from_experiment_name
import numpy as np
import itertools
from tqdm.notebook import tqdm


In [ ]:
assert settings.MLFLOW_TRACKING_URI


In [8]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)


In [9]:
ALL_CONFIDENCES = [0.1, 0.2, 0.3, 0.9]
ALL_WEIGHT_DECAYS = [0.1, 0.2, 0.3, 0.4]
ALL_MAX_LRS = [0.1, 0.2, 0.3, 0.4]
ALL_DATSETS = ["4-50", "5-50", "6-75"]
MODELS = ["resnet18", "resnet34", "resnet50", "vit"]


In [10]:
CONFIDENCES = [0.1,]
WEIGHT_DECAYS = [0.0, 0.1, 0.2, 0.3, 0.4]
MAX_LRS = [0.4]
MOMENTUMS = [0.1]
DATASETS = ["4-50"]
MODELS = ["vit"]

EXPERIMENT_NAMES = get_all_experiments(
    datasets=DATASETS,
    models=MODELS,
    confidences=CONFIDENCES,
    weight_decays=WEIGHT_DECAYS,
    max_lrs=MAX_LRS,
    momentums=MOMENTUMS,
)
EXPERIMENT_NAMES


['D4-50MvitC0.1WD0.0LR0.4M0.1',
 'D4-50MvitC0.1WD0.1LR0.4M0.1',
 'D4-50MvitC0.1WD0.2LR0.4M0.1',
 'D4-50MvitC0.1WD0.3LR0.4M0.1',
 'D4-50MvitC0.1WD0.4LR0.4M0.1']

In [11]:
experiments = [
    client.get_experiment_by_name(experiment_name)
    for experiment_name in tqdm(EXPERIMENT_NAMES)
]


  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
experiments


[<Experiment: artifact_location='mlflow-artifacts:/53', creation_time=1669514882888, experiment_id='53', last_update_time=1669514882888, lifecycle_stage='active', name='D4-50MvitC0.1WD0.0LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/35', creation_time=1669082388870, experiment_id='35', last_update_time=1669082388870, lifecycle_stage='active', name='D4-50MvitC0.1WD0.1LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/36', creation_time=1669082390041, experiment_id='36', last_update_time=1669082390041, lifecycle_stage='active', name='D4-50MvitC0.1WD0.3LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/37', creation_time=1669082391166, experiment_id='37', last_update_time=1669082391166, lifecycle_stage='active', n

In [11]:
def get_experiment_error_df(experiment_id: str) -> pd.DataFrame:
    src_errors = []
    tgt_errors = []
    runs = client.search_runs(experiment_ids=[experiment_id])
    for run in tqdm(runs):
        N = int(run.data.params["Frames"])
        filename = run.data.params["Filename"]
        src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
        tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
        src_error = 100 * (N - src_acc_top1) / N
        tgt_error = 100 * (N - tgt_acc_top1) / N
        assert src_error >= 0, (src_error, src_acc_top1, N, filename)
        assert tgt_error >= 0, (tgt_error, tgt_acc_top1, N, filename)
        src_errors.append(src_error)
        tgt_errors.append(tgt_error)

    df = pd.DataFrame(
        {
            "Source Error": src_errors,
            "Target Error": tgt_errors,
        }
    )
    return df.describe()


In [12]:
experiments

[<Experiment: artifact_location='mlflow-artifacts:/53', creation_time=1669514882888, experiment_id='53', last_update_time=1669514882888, lifecycle_stage='active', name='D4-50MvitC0.1WD0.0LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/35', creation_time=1669082388870, experiment_id='35', last_update_time=1669082388870, lifecycle_stage='active', name='D4-50MvitC0.1WD0.1LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/36', creation_time=1669082390041, experiment_id='36', last_update_time=1669082390041, lifecycle_stage='active', name='D4-50MvitC0.1WD0.3LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/37', creation_time=1669082391166, experiment_id='37', last_update_time=1669082391166, lifecycle_stage='active', n

In [13]:
dfs = [
    get_experiment_error_df(experiment.experiment_id)
    for experiment in tqdm(experiments)
]


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

In [14]:
experiment_names = [experiment.name for experiment in experiments]


In [15]:
params_df = pd.DataFrame(
    [
        get_params_from_experiment_name(experiment_name)
        for experiment_name in experiment_names
    ],
    index=experiment_names,
)
params_df

,dataset,model,confidence,weight_decay,max_lr,momentum
D4-50MvitC0.1WD0.0LR0.4M0.1,4-50,vit,0.1,0.0,0.4,0.1
D4-50MvitC0.1WD0.1LR0.4M0.1,4-50,vit,0.1,0.1,0.4,0.1
D4-50MvitC0.1WD0.2LR0.4M0.1,4-50,vit,0.1,0.2,0.4,0.1
D4-50MvitC0.1WD0.3LR0.4M0.1,4-50,vit,0.1,0.3,0.4,0.1
D4-50MvitC0.1WD0.4LR0.4M0.1,4-50,vit,0.1,0.4,0.4,0.1


In [16]:
mean_df = pd.concat([df.loc["mean"] for df in dfs], axis=1).T
mean_df = mean_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
mean_df = mean_df.rename(columns={"Source Error": "Source Error Mean", "Target Error": "Target Error Mean"})
mean_df_base = mean_df[["Source Error Mean", "Target Error Mean"]]
mean_df


,Source Error Mean,Target Error Mean
D4-50MvitC0.1WD0.0LR0.4M0.1,59.351015,50.589139
D4-50MvitC0.1WD0.1LR0.4M0.1,59.351015,52.489754
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648
D4-50MvitC0.1WD0.3LR0.4M0.1,59.351015,54.976481
D4-50MvitC0.1WD0.4LR0.4M0.1,59.351015,55.667846


In [17]:
mean_df["Error Mean Percentual Change"] = (
    100
    * (mean_df["Target Error Mean"] - mean_df["Source Error Mean"])
    / mean_df["Source Error Mean"]
)
# mean_df["Error Mean Absolute Change"] = mean_df["Target Error Mean"] - mean_df["Source Error Mean"]
mean_df_change = mean_df[["Error Mean Percentual Change"]]
mean_df


,Source Error Mean,Target Error Mean,Error Mean Percentual Change
D4-50MvitC0.1WD0.0LR0.4M0.1,59.351015,50.589139,-14.762807
D4-50MvitC0.1WD0.1LR0.4M0.1,59.351015,52.489754,-11.560478
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124
D4-50MvitC0.1WD0.3LR0.4M0.1,59.351015,54.976481,-7.370614
D4-50MvitC0.1WD0.4LR0.4M0.1,59.351015,55.667846,-6.205738


In [18]:
std_df = pd.concat([df.loc["std"] for df in dfs], axis=1).T
std_df = std_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
std_df = std_df.rename(columns={"Source Error": "Source Error Std", "Target Error": "Target Error Std"})
std_df_base = std_df[["Source Error Std", "Target Error Std"]]
std_df

,Source Error Std,Target Error Std
D4-50MvitC0.1WD0.0LR0.4M0.1,23.749646,27.629513
D4-50MvitC0.1WD0.1LR0.4M0.1,23.749646,26.694260
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293
D4-50MvitC0.1WD0.3LR0.4M0.1,23.749646,25.581194
D4-50MvitC0.1WD0.4LR0.4M0.1,23.749646,25.259107


In [19]:
std_df["Error Std Percentual Change"] = (
    100
    * (std_df["Target Error Std"] - std_df["Source Error Std"])
    / std_df["Source Error Std"]
)
# std_df["Error Std Absolute Change"] = std_df["Target Error Std"] - std_df["Source Error Std"]
std_df_change = std_df[["Error Std Percentual Change"]]
std_df


,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.1WD0.0LR0.4M0.1,23.749646,27.629513,16.336525
D4-50MvitC0.1WD0.1LR0.4M0.1,23.749646,26.694260,12.398556
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293,9.716553
D4-50MvitC0.1WD0.3LR0.4M0.1,23.749646,25.581194,7.711893
D4-50MvitC0.1WD0.4LR0.4M0.1,23.749646,25.259107,6.355718


In [20]:
mean_df.join(std_df)

,Source Error Mean,Target Error Mean,Error Mean Percentual Change,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.1WD0.0LR0.4M0.1,59.351015,50.589139,-14.762807,23.749646,27.629513,16.336525
D4-50MvitC0.1WD0.1LR0.4M0.1,59.351015,52.489754,-11.560478,23.749646,26.694260,12.398556
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124,23.749646,26.057293,9.716553
D4-50MvitC0.1WD0.3LR0.4M0.1,59.351015,54.976481,-7.370614,23.749646,25.581194,7.711893
D4-50MvitC0.1WD0.4LR0.4M0.1,59.351015,55.667846,-6.205738,23.749646,25.259107,6.355718


In [21]:
all_df = params_df[["weight_decay"]].join(mean_df).join(std_df)
all_df.to_latex(float_format="%.2f")


/tmp/ipykernel_32/3637889511.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  all_df.to_latex(float_format="%.2f")


'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &  weight\\_decay &  Source Error Mean &  Target Error Mean &  Error Mean Percentual Change &  Source Error Std &  Target Error Std &  Error Std Percentual Change \\\\\n\\midrule\nD4-50MvitC0.1WD0.0LR0.4M0.1 &          0.00 &              59.35 &              50.59 &                        -14.76 &             23.75 &             27.63 &                        16.34 \\\\\nD4-50MvitC0.1WD0.1LR0.4M0.1 &          0.10 &              59.35 &              52.49 &                        -11.56 &             23.75 &             26.69 &                        12.40 \\\\\nD4-50MvitC0.1WD0.2LR0.4M0.1 &          0.20 &              59.35 &              53.89 &                         -9.20 &             23.75 &             26.06 &                         9.72 \\\\\nD4-50MvitC0.1WD0.3LR0.4M0.1 &          0.30 &              59.35 &              54.98 &                         -7.37 &             23.75 &             25.58 &                         7.71 \\\

In [22]:
all_df_base = params_df[["weight_decay"]].join(mean_df_base).join(std_df_base)
all_df_base

,weight_decay,Source Error Mean,Target Error Mean,Source Error Std,Target Error Std
D4-50MvitC0.1WD0.0LR0.4M0.1,0.0,59.351015,50.589139,23.749646,27.629513
D4-50MvitC0.1WD0.1LR0.4M0.1,0.1,59.351015,52.489754,23.749646,26.694260
D4-50MvitC0.1WD0.2LR0.4M0.1,0.2,59.351015,53.890648,23.749646,26.057293
D4-50MvitC0.1WD0.3LR0.4M0.1,0.3,59.351015,54.976481,23.749646,25.581194
D4-50MvitC0.1WD0.4LR0.4M0.1,0.4,59.351015,55.667846,23.749646,25.259107


In [23]:
all_df_change = params_df[["weight_decay"]].join(mean_df_change).join(std_df_change)
all_df_change

,weight_decay,Error Mean Percentual Change,Error Std Percentual Change
D4-50MvitC0.1WD0.0LR0.4M0.1,0.0,-14.762807,16.336525
D4-50MvitC0.1WD0.1LR0.4M0.1,0.1,-11.560478,12.398556
D4-50MvitC0.1WD0.2LR0.4M0.1,0.2,-9.200124,9.716553
D4-50MvitC0.1WD0.3LR0.4M0.1,0.3,-7.370614,7.711893
D4-50MvitC0.1WD0.4LR0.4M0.1,0.4,-6.205738,6.355718


In [1]:
def find_worse(better_id: str, worse_id: str):
    runs = client.search_runs(experiment_ids=[better_id, worse_id], max_results=2000)
    ekey = lambda run: run.info.experiment_id
    runs.sort(key=ekey)

    experiment_runs = dict()
    for k, group in itertools.groupby(runs, key=ekey):
        experiment_runs[k] = {run.data.params["Filename"]: run for run in group}

    metric = "Target Accumulated Top-1 Count"
    worsened = []
    for filename, run in experiment_runs[worse_id].items():
        if (
            experiment_runs[better_id][filename].data.metrics[metric]
            < run.data.metrics[metric]
        ):
            worsened.append(filename)
    
    rows = []
    for filename in worsened:
        baseline = "Source Accumulated Top-1 Count"
        metric = "Target Accumulated Top-1 Count"

        row = {
            "Filename": filename,
            baseline: experiment_runs[worse_id][filename].data.metrics[baseline],
            f"{worse_id}-{metric}": experiment_runs[worse_id][filename].data.metrics[metric],
            f"{better_id}-{metric}": experiment_runs[better_id][filename].data.metrics[metric],
        }
        rows.append(row)

    return pd.DataFrame(rows)
            

In [15]:
df = find_worse(better_id="53", worse_id="32")

In [16]:
df

,Filename,Source Accumulated Top-1 Count,32-Target Accumulated Top-1 Count,53-Target Accumulated Top-1 Count
0,/datasets/imagenet1k-4-50/979-valley-4-40.mp4,24.0,20.0,19.0
1,/datasets/imagenet1k-4-50/946-cardoon-4-40.mp4,32.0,32.0,28.0
2,/datasets/imagenet1k-4-50/917-comic_book-4-40.mp4,8.0,10.0,8.0
3,/datasets/imagenet1k-4-50/871-trimaran-4-40.mp4,26.0,20.0,2.0
4,/datasets/imagenet1k-4-50/831-studio_couch-4-4...,12.0,13.0,12.0
5,/datasets/imagenet1k-4-50/829-streetcar-4-40.mp4,12.0,11.0,9.0
6,/datasets/imagenet1k-4-50/753-radiator-4-40.mp4,12.0,12.0,11.0
7,/datasets/imagenet1k-4-50/748-purse-4-40.mp4,2.0,2.0,0.0
8,/datasets/imagenet1k-4-50/74-garden_spider-4-4...,4.0,1.0,0.0
9,/datasets/imagenet1k-4-50/735-poncho-4-40.mp4,8.0,2.0,0.0
